# Library

In [1]:
import glob
import os
import pandas as pd
import pandas as pd
import seaborn as sns
import datetime as dt
import mysql.connector
import matplotlib.pyplot as plt
import plotly.express as px

# Import Data

In [ ]:
mydb = mysql.connector.connect(
  host="host",
  user="user",
  password="pass",
  database="gdb",
  port = 0000
)

In [3]:
mycursor = mydb.cursor()

In [ ]:
mycursor.execute(
"""
query
"""
)

In [ ]:
result = mycursor.fetchall()

In [ ]:
df_db = pd.DataFrame(result, columns= ["id","invoice_date","product_name","product_sales_total","tracking_number"])
df_db

## Return Data

In [ ]:
directory = r'C:\Users\linkc\py\GB\Ref\return'
all_files = glob.glob(os.path.join(directory, "*.csv"))

In [ ]:
dataframes = []

column_awb = ["AWB", "AWB#", "Tracking Id"]

for file in all_files:
    df = pd.read_csv(file)
    
    for col in column_awb:
        if col in df.columns:
            df = df.rename(columns={col: 'AWB'})
            break 

    dataframes.append(df)
    
return_dataset = pd.concat(dataframes, ignore_index=True)
return_dataset

In [ ]:
df_db['tracking_number_str'] = df_db['tracking_number'].astype(str)
return_dataset['AWB'] = return_dataset['AWB'].astype(str)

In [ ]:
dataset = df_db[~df_db['tracking_number_str'].isin(return_dataset['AWB'])]
dataset

### Remove Invalid Date

In [ ]:
df_db  = df_db[df_db['invoice_date'] != '0000-00-00']
# df_db

# Filtering Product

In [ ]:
df_db['invoice_date'] = pd.to_datetime(df_db['invoice_date'])
df_db['invoice_date'] = df_db['invoice_date'].dt.strftime('%Y-%m-%d')
df_db = df_db[df_db['product_name'] == 'PRODUCT']
# df_db

In [ ]:
df_db['invoice_date'] = pd.to_datetime(df_db['invoice_date'])
df_db['invoice_month'] = df_db['invoice_date'].dt.to_period('M').dt.to_timestamp()

In [ ]:
grouping = df_db.groupby('id')['invoice_month']
df_db['cohort_month'] = grouping.transform('min')
df_db

# Cohort Index

In [ ]:
def get_date_int(df_db, column):
    year = df_db[column].dt.year
    month = df_db[column].dt.month
    date = df_db[column].dt.date
    return year, month, date

Invoice_Year, Invoice_Month, _ = get_date_int(df_db, 'invoice_month')
Cohort_Year, Cohort_Month , _ = get_date_int(df_db, 'cohort_month')
Year_Diff = Invoice_Year - Cohort_Year
Month_Diff = Invoice_Month - Cohort_Month
df_db['cohort_index'] = Year_Diff*12 + Month_Diff +1
df_db.head()

# Cohort

In [ ]:
grouping = df_db.groupby(['cohort_month', 'cohort_index'])
monet_data = grouping['product_sales_total'].sum().reset_index()
monet_data.head()

In [ ]:
monet_mon = monet_data.pivot(index='cohort_month', columns='cohort_index', values='product_sales_total')
monet_mon

In [ ]:
file_path = 'dir/file.csv' 
monet_mon.to_csv(file_path, index=True)

In [ ]:
monet_sizes = monet_mon.iloc[:, 0]
monet_percentage = monet_mon.divide(monet_sizes, axis=0)
monet_percentage

In [ ]:
file_path_percent = 'dir/file.csv.csv'
monet_percentage.to_csv(file_path_percent, index=True)